In [1]:
import os, sys


filedir = os.path.abspath(".")

currentdir = os.path.dirname(os.path.abspath(".")) # workaround to adding local modules

parentdir = os.path.dirname(os.path.abspath(""+currentdir))
print(f'CurrentDir: {currentdir}')
print(f'ParentDir: {parentdir}')

sys.path.insert(0, currentdir)

CurrentDir: g:\Meu Drive\Estudos\Pós Graduação\MESTRADO\projeto\git\multimorbity_use
ParentDir: g:\Meu Drive\Estudos\Pós Graduação\MESTRADO\projeto\git


# Importing Libraries

In [2]:
import pandas as pd
import numpy as np
import os.path as osp

from src.config import *

pd.options.display.max_columns = None
pd.options.display.float_format = "{:,.2f}".format

# Definitions

In [3]:
DATA_PATH = osp.join('..', 'data', 'raw','dataset_follow_up.xlsx')
OUTPUT_PATH = osp.join('..', 'data', 'processed','dataset_clean.csv')
TARGET = 'pronto_atendimento'


# Reading Data

In [4]:
df = pd.read_excel(DATA_PATH, sheet_name = 0, header = 0)

df.sample(10)


record_id         dt      hora                  entrev   
4315      12414 2021-11-19  13:58:00       ana clara santana  \
2827       9122 2021-10-25  13:55:00        Danielle Campelo   
550        2255 2021-09-11  16:02:00       ana clara santana   
2336       7930 2021-10-13  16:08:00  ana paula krause lopes   
3587      10860 2021-11-07  17:06:00       ana clara santana   
1570       4736 2021-09-27  17:12:00            Larissa rios   
929        3005 2021-09-16  13:50:00            Larissa rios   
4778      13655 2021-11-28  14:30:00            Larissa rios   
1063       3540 2021-09-18  14:34:00         Miriam Dallmann   
394        1259 2021-09-09  16:01:00           Tatiana porto   

                              setor domicilio endpdr  idaderc  sexorc  recusa   
4315      431440705150011 - FRAGATA  Elegível    NaN      NaN     NaN     NaN  \
2827  431440705130020 - TRÊS VENDAS  Elegível    NaN      NaN     NaN     NaN   
550   431440705130033 - TRÊS VENDAS  Elegível    NaN      NaN     NaN     NaN   
2336  431440705130015 - TRÊS VENDAS  Elegível    NaN      NaN     NaN     NaN   
3587       431440705150044, FRAGATA  Elegível    NaN      NaN     NaN     NaN   
1570       431440705160115 - CENTRO  Elegível    NaN      NaN     NaN     NaN   
929       431440705150097 - FRAGATA  Elegível    NaN      NaN     NaN     NaN   
4778      431440705150081 - FRAGATA  Elegível    NaN      NaN     NaN     NaN   
1063      431440705150100 - FRAGATA  Elegível    NaN      NaN     NaN     NaN   
394   431440705190001 - LARANJAL/Z3  Elegível    NaN      NaN     NaN     NaN   

      recusaout                   latitude                  longitude   
4315        NaN  -3,175,157,174,198,660.00  -5,237,088,280,178,190.00  \
2827        NaN -31,707,488,335,446,600.00  -5,235,710,707,952,910.00   
550         NaN                        NaN                        NaN   
2336        NaN            -317,171,262.00            -523,487,089.00   
3587        NaN -31,763,758,660,902,800.00 -52,366,957,713,048,000.00   
1570        NaN  -3,177,453,329,600,340.00 -52,335,734,374,316,704.00   
929         NaN                        NaN                        NaN   
4778        NaN    -317,420,954,313,272.00  -5,237,147,176,814,840.00   
1063        NaN            -317,542,783.00            -523,723,292.00   
394         NaN                        NaN                        NaN   

                                  nmor   sexomor1  idademor1   
4315                                 2   Feminino         74  \
2827                                 3   Feminino         28   
550   1 (apenas o próprio respondente)   Feminino         63   
2336                                 2   Feminino         53   
3587  1 (apenas o próprio respondente)  Masculino         54   
1570                                 2  Masculino         61   
929                                  2  Masculino         18   
4778                                 3   Feminino         36   
1063  1 (apenas o próprio respondente)   Feminino         48   
394                                  2   Feminino         53   

                   trabmor1 atforamor1   sexomor2  idademor2   
4315          Aposentado(a)        NaN  Masculino      24.00  \
2827               Trabalha        Sim  Masculino      32.00   
550   Em auxilio do governo        NaN        NaN        NaN   
2336                    Não        NaN   Feminino      83.00   
3587               Trabalha        Sim        NaN        NaN   
1570          Aposentado(a)        NaN   Feminino      62.00   
929                Trabalha        Não  Masculino      18.00   
4778                    Não        NaN  Masculino      18.00   
1063                    Não        NaN        NaN        NaN   
394                Trabalha        Não  Masculino      51.00   

               trabmor2 atforamor2   sexomor3  idademor3 trabmor3 atforamor3   
4315             Estuda        NaN        NaN        NaN      NaN        NaN  \
2827  Trabalha e estuda        NaN 

# Procesing columns

In [5]:
# # !pip install unidecode
# import unidecode

# def standardize_column_name(col):
#     col = unidecode.unidecode(col)
#     return col.replace(' ', '_').lower()

# df.columns = [standardize_column_name(col) for col in df.columns]

# Data Cleaning

## Filtering Rows and Columns

In [6]:
print(f'df.shape pre-filter: {df.shape}')

# Filtering only matched rows (Baseline + Follow-up)
pdf = df.query("_merge == 'Matched (3)'")

# Get follow-up features to avoid data-leakage
follow_up_columns = df.columns[pdf.columns.get_loc('ACOMPANHAMENTO'):].to_list()

# Calculating target feature
pdf['pronto_atendimento'] = ((~pdf[pronto_atendimento].isin(['Não utilizou'])).sum(axis=1) > 0).astype(int)

# Drop unwanted columns

pdf = pdf.drop(drop_features + follow_up_columns, axis = 1)

print(f'pdf.shape post-filter: {pdf.shape}')

df.shape pre-filter: (5722, 875)
pdf.shape post-filter: (3502, 381)


C:\Users\renan\AppData\Local\Temp\ipykernel_10208\1925275048.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pdf['pronto_atendimento'] = ((~pdf[pronto_atendimento].isin(['Não utilizou'])).sum(axis=1) > 0).astype(int)


## Treating missing rows

In [7]:
# Drop rows with all missing values
pdf.dropna(how = 'all', inplace = True)

# Feature Engineering

In [8]:
# Calculating multimorbidity
pdf['multi_calc'] = pdf[morbidades].isin(['sim',1,'Sim','1']).sum(axis=1)

# Adjusting Dtypes

In [9]:
# Apply dtypes
pdf = pdf.astype(dtypes_dict)

## Preprocessing

In [10]:
# dropping missing values on the target
pdf.dropna(subset=[TARGET], inplace=True)

In [11]:
# defining categorical features
# df[categorical_features] = df[categorical_features].astype(str)

# defining numeric features
# df[numeric_features] = df[numeric_features].astype('float')

# defining o target
pdf[TARGET] = pdf[TARGET].astype('int')

# Final DataFrame

In [12]:
pdf.sample(10)

,setor,nmor,genero,cor,ler,escol,sit,altura,peso,conj,trab,rendapd,fumo,tafumo,tmfumo,tpafumo,tpmfumo,cigdia,audit1,audit2,audit3,audit4,audit5,audit6,audit7,audit8,audit9,audit10,abmicro,CW,ablav,abgel,abfree,ablavr,tv,tvcabo,abdvd,abmco,ac,abmoto,abauto,abseca,pecas,abban,abemd,abag,abpav,vz,md,abesc,renda,ajudarenda,ajudacovid___0,ajudacovid___1,ajudacovid___2,ajudacovid___3,ajudacovid___9,alirenda,cmdac,cmddn,dnali,dmali,mcom,cam,camt,aff,afft,afm,afmt,dcam,dcamt,dbici,dbicit,exer,exert,sent,deit,tvs,ppaf,atd,uatd,atdpt,jnprecisou,jnbuscou,jnconseguiu,jnondebuscou,jnmotivonao,plano,qplan,medenf,tamenf,tmmenf,imedenf,imedenfout,pref,postoqual,hmed,hmedout,nserv,tserv,tservout,eserv,eservout,introcovid,qcovid,tratcovid,rtestefam,tratcovidfam,covidfam,covidamg,vacina,dtvac,dtvac2,mvac,mvacout,vgp0,vgp1,cdt,ldt,mdt,omdt,autsa,comsa,idsa,sat,qsono,tsono,dcosta,bsec,asm,brq,enfs,dpoc,tpul___1,tpul___2,tpul___3,tpul___4,tpul___5,tpul___6,chiado,nmed,medind,medd,aqmed,inaf,ost,art,has,diab,pcor,park,rin,tire,glau,cat,alzh,inc,hepat,dor,out,outqual,dgp,qdgp___1,qdgp___2,qdgp___3,qdgp___4,qdgp___5,qdgp___6,qdgp___7,qdgp___8,qdgp___9,qdgp___10,qdgp___11,qdgp___12,qdgp___13,qdgp___14,qdgp___15,qdgp___16,qdgp___17,qdgp___18,qdgp___19,ang,avc,vavc,colt,epil,vepil,dep,ulc,itu,vitu,flu,vflu,pnm,vpnm,ca,taca,tmca,lca,ttca,pfarm,hemo,qtp,rtp,outt,qoutt,qued,vqued,acam,taacam,tmacam,inth,intht,poucoint9,pbaixo9,difsono9,cans9,semapetite9,sentiumal9,difconc9,lento9,ferir9,diftrab9,nerv7,imped7,preocup7,relax7,agit7,abor7,medo7,ajucama,ajumed,ajutare,ajuref,pessoamt,amor,abraco,queame,pessoamta,ouvir,confiar,compart,comprob,pessoaem,conselho,infajuda,quercons,sugprobm,pessoainf,fazagrad,discab,relax,divertir,pessoasp,ppeso,fraco,devg,menosaf,difcoisa,esfo,atvpd,banho,vestir,cama,lugares,banheiro,alimentar,cappd,mcj,dge,dsp,vlc,sozi,serv,idag,qserv,conj3,cor4,estudo4cat,cancer,fumo2,alc2,peso1,altura1,bmi,bmi4cat,obesidade,rendasetor,rensetor10,urg,rensetor5,_Irensetor1_2,_Irensetor1_3,_Irensetor1_4,_Irensetor1_5,_Irensetor1_6,_Irensetor1_7,_Irensetor1_8,_Irensetor1_9,_Irensetor1_10,banheiroabep,empabep,carroabep,microabep,lavaloucabep,gelaabep,freeabep,lavaroupaabep,dvdabep,microondasabep,motoabep,secaabep,chefeabep,aguaabep,pavabep,abep,abepcat,esc4j,sexj,cor2,abep4,j,ds,esperado,total,id4,mor2,idibge,pw,pw1,pop,estrato,idade,icovid,depre1,depre2,depre3,depre4,depre5,depre6,depre7,depre8,depre9,depress,depre_algo,depresscat,phq_sum_b,depre_cut,ansi1,ansi2,ansi3,ansi4,ansi5,ansi6,ansi7,ansiss,ansi_algo,gad_sum_b,ansi_cut,gad_sintomas,gad_severity,gad_severo,phq_severity,popw,is,id2,pros,multi,m2,m3,hospit,cmdac1,cmddn1,dnali1,dmali1,mcom1,inseg,testecovid,tcovid,asmbrq,pronto_atendimento,multi_calc
2866,431440705130023 - TRÊS VENDAS,3,Homem cisgênero (1),Branca,sim,8º série do 1º grau (antigo ginásio),Trabalhando,165,74.00,casado(a) ou com companheiro(a),sim,Piorou muito,"não, nunca fumou",NaN,NaN,NaN,NaN,NaN,duas a quatro vezes por mês,uma ou duas,menos de uma vez por mês,nunca,nunca,nunca,nunca,nunca,não,não,"Sim, uma",1,Não possui,"Sim, uma","Sim, uma","Sim, uma","Sim, duas",Não,Não possui,"Sim, uma",Não possui,Não possui,"Sim, uma",Não possui,2.00,1.00,nenhuma,Rede geral de distribuição,Terra/Cascalho,Segura,Sim,Fundamental I completo / Fundamental II incomp...,Diminuiu muito,Não,Checked,Unchecked,Unchecked,Unchecked,Unchecked,Não,Não,Não,Não,Não,Não,Nenhum,NaN,Nenhum,NaN,Nenhum,NaN,Nenhum,NaN,Nenhum,NaN,Não,NaN,300.00,0.00,120.00,Não,Não,NaN,NaN,Não,Não,NaN,NaN,NaN,sim,Cartão de todos,não,NaN,NaN,NaN,NaN,não,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,Sim,Marco,Domicílio,Sim,Domicílio,Não,Não,Sim,Julho,Setembro,CoronaVac/Butantan,NaN,Não,Não,Nos últimos 6 meses,Serviço Público,Rotina/manutenção,NaN,Boa,Mesma coisa,Igual,8.00,Boa,7.00,Não,"Nunca,",Não,Não,Não,Não,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,NaN,NaN,NaN,NaN,NaN,NaN,Não,Não,Sim,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,NaN,Não,Unchecked,Unchecked,Unchecked,Unchecke

# Saving New DataFrame

In [13]:
pdf.to_csv(OUTPUT_PATH, index = False, header = True)